In [1]:
import math
import numpy as np
import pandas as pd
import os

In [2]:
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
with open("Results/constants.txt") as f:
    str = f.readline()
    while str:
        if str.startswith("Mean Semi") == True:
            semi_major_axis = np.float128(str.split(':')[1].split(' ')[1].strip())
        elif str.startswith("Eccentricity") == True:
            e = np.float128(str.split(':')[1].split(' ')[1].strip())
        elif str.startswith("Mean Inclination") == True:
            inc = np.float128(str.split(':')[1].split(' ')[1].strip())
        elif str.startswith("Mean Longitude") == True:
            omega = np.float128(str.split(':')[1].split(' ')[1].strip())
        elif str.startswith("Mean Argument") == True:
            w = np.float128(str.split(':')[1].split(' ')[1].strip())

        str = f.readline()



In [4]:
T = 225 #Time period of Venus being 224.7 Earth days
pred_t = np.arange(0,T,1)
mean_motion = 2*np.pi/T

mean_anomaly = mean_motion*pred_t

In [5]:
E = mean_anomaly
E_nxt = E - (E - e*np.sin(E) - mean_anomaly)/(1 - e*np.cos(E))
i = 0
while not(np.all(np.abs(E - E_nxt) < 0.5*(10**-7))):
    i+=1
    E = E_nxt
    E_nxt = E - (E - e*np.sin(E) - mean_anomaly)/(1 - e*np.cos(E))
E = E_nxt

In [6]:
true_anomaly = 2*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2))
# #adjust quadrant
true_anomaly = np.where(true_anomaly<0, true_anomaly+2*math.pi, true_anomaly)

In [7]:
radius = semi_major_axis*(1 - e**2)/(1 + e*np.cos(true_anomaly))

In [8]:
#perifocal coordinates
x = radius*np.cos(true_anomaly)
y = radius*np.sin(true_anomaly)
z = np.zeros_like(x)

In [9]:
#rotation matrix
R = np.array([
        [np.cos(w)*np.cos(omega) - np.sin(w)*np.cos(inc)*np.sin(omega), -np.cos(w)*np.sin(omega) - np.sin(w)*np.cos(inc)*np.cos(omega), np.sin(w)*np.sin(inc)],
        [np.sin(w)*np.cos(omega) + np.cos(w)*np.cos(inc)*np.sin(omega), -np.sin(w)*np.sin(omega) + np.cos(w)*np.cos(inc)*np.cos(omega), -np.cos(w)*np.sin(inc)],
        [np.sin(inc)*np.sin(omega), np.sin(inc)*np.cos(omega), np.cos(inc)]
    ])

#rotate
r = np.array([x,y,z])
r = np.dot(R,r)

In [10]:
#write coordinates to a file
with open("Results/venus_2023.txt", "w") as f:
    for i in range(len(r[0])):
        f.write("{:.20f}".format(r[0][i]) + " " + "{:.20f}".format(r[1][i]) + " " + "{:.20f}".format(r[2][i]) + "\n")